# Import Library

In [ ]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.3 MB/s 
     |████████████████████████████████| 1.6 MB 20.9 MB/s 


# Load Existing Data

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from kerastuner.tuners import RandomSearch
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import joblib
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import confusion_matrix,accuracy_score
from matplotlib import pyplot
import time

X_train = pd.read_excel('X_train.xlsx').drop(columns=['Unnamed: 0'])
X_test = pd.read_excel('X_test.xlsx').drop(columns=['Unnamed: 0'])
y_train = pd.read_excel('y_train.xlsx').drop(columns=['Unnamed: 0'])
y_test = pd.read_excel('y_test.xlsx').drop(columns=['Unnamed: 0'])

<ipython-input-2-9969d3e83569>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

In [ ]:
X_train.shape

(106, 1, 27)

# LSTM Hyperparameter

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(hp.Choice('LSTM_unit_1', [2,4,8,16,32]), return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(hp.Float('Dropout_rate_1',min_value=0,max_value=0.5,step=0.1)))
    model.add(LSTM(hp.Choice('LSTM_unit_2', [2,4,8,16,32]), return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(hp.Float('Dropout_rate_2',min_value=0,max_value=0.5,step=0.1)))
    model.add(LSTM(hp.Choice('LSTM_unit_3', [2,4,8,16,32])))
    model.add(Dropout(hp.Float('Dropout_rate_3',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(1, activation=hp.Choice('dense_activation',values=['sigmoid'],default='sigmoid')))
    model.compile(loss='binary_crossentropy', optimizer=hp.Choice('optimizer', ['adam', 'adamax', 'sgd']), metrics=['binary_accuracy'])
    return model

In [ ]:
tuner= RandomSearch(
        build_model,
        objective='val_binary_accuracy',
        max_trials=20,
        executions_per_trial=1
        )

In [ ]:
tuner.search(
        x=X_train,
        y=y_train,
        validation_data=(X_test, y_test), 
        epochs=500, 
        batch_size=32
        #, callbacks=[es, mc]
)

Trial 20 Complete [00h 00m 49s]
val_binary_accuracy: 0.6111111044883728

Best val_binary_accuracy So Far: 0.7777777910232544
Total elapsed time: 00h 18m 08s


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 8
LSTM_unit_1 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 4, 8, 16, 32], 'ordered': True}
Dropout_rate_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
LSTM_unit_2 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 4, 8, 16, 32], 'ordered': True}
Dropout_rate_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
LSTM_unit_3 (Choice)
{'default': 2, 'conditions': [], 'values': [2, 4, 8, 16, 32], 'ordered': True}
Dropout_rate_3 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
dense_activation (Choice)
{'default': 'sigmoid', 'conditions': [], 'values': ['sigmoid'], 'ordered': False}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'adamax', 'sgd'], 'ordered': False}


In [ ]:
tuner.results_summary(num_trials = 10)

Results summary
Results in ./untitled_project
Showing 50 best trials
Trial summary
Hyperparameters:
LSTM_unit_1: 32
Dropout_rate_1: 0.5
LSTM_unit_2: 4
Dropout_rate_2: 0.2
LSTM_unit_3: 8
Dropout_rate_3: 0.2
dense_activation: sigmoid
optimizer: adam
Score: 0.7777777910232544
Trial summary
Hyperparameters:
LSTM_unit_1: 4
Dropout_rate_1: 0.30000000000000004
LSTM_unit_2: 2
Dropout_rate_2: 0.5
LSTM_unit_3: 32
Dropout_rate_3: 0.0
dense_activation: sigmoid
optimizer: adam
Score: 0.7222222089767456
Trial summary
Hyperparameters:
LSTM_unit_1: 32
Dropout_rate_1: 0.2
LSTM_unit_2: 8
Dropout_rate_2: 0.1
LSTM_unit_3: 16
Dropout_rate_3: 0.2
dense_activation: sigmoid
optimizer: adamax
Score: 0.7222222089767456
Trial summary
Hyperparameters:
LSTM_unit_1: 2
Dropout_rate_1: 0.2
LSTM_unit_2: 4
Dropout_rate_2: 0.0
LSTM_unit_3: 8
Dropout_rate_3: 0.2
dense_activation: sigmoid
optimizer: adamax
Score: 0.7222222089767456
Trial summary
Hyperparameters:
LSTM_unit_1: 16
Dropout_rate_1: 0.1
LSTM_unit_2: 32
Dropout_

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]

In [ ]:
print("Lstm units 1   : ", best_hps.get('LSTM_unit_1'))
print("Dropout 1      : ", best_hps.get('Dropout_rate_1'))
print("Lstm units 2   : ", best_hps.get('LSTM_unit_2'))
print("Dropout 2      : ", best_hps.get('Dropout_rate_2'))
print("Lstm units 3   : ", best_hps.get('LSTM_unit_3'))
print("Dropout 3      : ", best_hps.get('Dropout_rate_3'))
print("Activation     : ", best_hps.get('dense_activation'))
print("Optimizer      : ", best_hps.get('optimizer'))

Lstm units 1   :  32
Dropout 1      :  0.5
Lstm units 2   :  4
Dropout 2      :  0.2
Lstm units 3   :  8
Dropout 3      :  0.2
Activation     :  sigmoid
Optimizer      :  adam


# LSTM Model

In [ ]:
model = tuner.hypermodel.build(best_hps)

# Early Stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

# Model Checkpoint
mc = ModelCheckpoint('best_model.h5', monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5000, batch_size=32, callbacks=[es, mc])

Epoch 1/5000
1/4 [======>.......................] - ETA: 20s - loss: 0.6934 - binary_accuracy: 0.5312
Epoch 1: val_binary_accuracy improved from -inf to 0.58333, saving model to best_model.h5
4/4 [==============================] - 9s 536ms/step - loss: 0.6924 - binary_accuracy: 0.5943 - val_loss: 0.6923 - val_binary_accuracy: 0.5833
Epoch 2/5000
1/4 [======>.......................] - ETA: 0s - loss: 0.6933 - binary_accuracy: 0.5312
Epoch 2: val_binary_accuracy improved from 0.58333 to 0.61111, saving model to best_model.h5
4/4 [==============================] - 0s 29ms/step - loss: 0.6919 - binary_accuracy: 0.5566 - val_loss: 0.6914 - val_binary_accuracy: 0.6111
Epoch 3/5000
1/4 [======>.......................] - ETA: 0s - loss: 0.6878 - binary_accuracy: 0.6875
Epoch 3: val_binary_accuracy did not improve from 0.61111
4/4 [==============================] - 0s 26ms/step - loss: 0.6895 - binary_accuracy: 0.6132 - val_loss: 0.6906 - val_binary_accuracy: 0.6111
Epoch 4/5000
1/4 [======>...

In [ ]:
# load the saved model
saved_model = load_model('best_model.h5')

In [ ]:
y_pred = saved_model.predict(X_test)

# print("Train Time: %f seconds" % traintime)
# print("Test Time: %s seconds" % (time.time() - start_time))

y_pred = pd.DataFrame({'y_pred': y_pred.round().ravel()})
# y_pred = list(map(_replaceitem, y_pred.ravel()))
results = pd.concat([y_test,y_pred], axis=1)

2/2 [==============================] - 1s 6ms/step


In [ ]:
if(len(confusion_matrix(y_test, y_pred).ravel()) > 1):
  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
  print(tn, fp, fn, tp)
else:
  cm = confusion_matrix(y_test, y_pred)
  print(cm)
ac = accuracy_score(y_test,y_pred)

7 7 3 19


In [ ]:
ac

0.7222222222222222